In [ ]:
## import serial
import time
from matplotlib import pyplot as plt
import csv
import os
import datetime
import random
import numpy as np
import serial.tools.list_ports
from matplotlib.ticker import NullFormatter

'''
¿configurar cuanto tiempo de grabación?
¿cada cuanto tiempo se realiza la lectura análogo?
¿como se guardan los datos y donde?
'''

puerto = seleccionar_puerto()
fecha = datetime.datetime.now().strftime("%d-%m-%Y_%H;%M;%S")
ruta_carpeta = r'/home/manuelm/Documentos/DLAB/Registro-PH/Software/data'
vector = []

#########################################################################################################

def seleccionar_puerto():
    puertos_disponibles = [puerto.device for puerto in serial.tools.list_ports.comports()]

    if not puertos_disponibles:
        print("No se encontraron puertos seriales disponibles.")
        return None

    print("SELECCIONE PUERTO SERIAL:")
    for indice, puerto in enumerate(puertos_disponibles, start=1):
        print(f"{indice}. {puerto}")

    seleccion = 0
    while True:
        try:
            seleccion = int(input("Seleccione el número del puerto a usar: "))
            print("------------------------------------------------")
            if 1 <= seleccion <= len(puertos_disponibles):
                break
            else:
                print("Seleccion incorrecta. Intente de nuevo.")
        except ValueError:
            print("Entrada inválida. Por favor, ingrese un número.")

    puerto_seleccionado = puertos_disponibles[seleccion - 1]
    return puerto_seleccionado

##########################################################################################################

def main():
    arduino = serial.Serial(puerto, 115200, timeout=1) #windows
    time.sleep(1) # Espera a que Arduino se inicialice
    arduino.flushInput()
    print("MENU PRINCIPAL")  
    print("1. Iniciar Grabacion de datos")
    print("2. Salir")
    while True:
        opcionn = input("Seleccione una opción: ")
        
        if opcionn == '1':
            t_grabacion = input("Seleccione el tiempo total de grabacion")
            while True:
                t_lectura = input("Seleccione un intervalo de tiempo (segs) entre lecturas:")
                if int(t_lectura)>1000:
                    print("Error, lectura fuera de rango")
                else:
                    break
                while True:
                    iniciar = str(input("Quiere iniciar el programa? (si / no):"))
                    if iniciar =="si":
                        print("iniciando el programa...")
                        instruccion = str(opcionn)+','+str(t_grabacion)+','+str(t_lectura)+','+str(iniciar)+'\r'
                        print("Enviando a Arduino: ", instruccion)
                        time.sleep(1)
                        arduino.write(instruccion.encode('utf-8'))
                        time.sleep(1)
                        response = arduino.readline().decode().strip()
                        print("Respuesta de Arduino:", response)
                        contador = 0

                        while True:
                            if arduino.in_waiting > 0:
                                serial_data = arduino.readline().decode().rstrip()
                                vector.append(int(serial_data))                   
                                else:
                                    if len(vector)==(int(Nveces_adc)-1):
                                        #print(vector)
                                        vector_m = [x * 3.3 for x in vector]
                                        vector_voltaje = [round(x / 255, 2) for x in vector_m]
                                        print(vector_voltaje)
                                        tiempo = np.arange(0, len(vector), 1)
                                        #tiempo = np.arange(0, len(vector) * 3, 3)
                                        print("MENU PLOTEO")
                                        print("1. Plot datos brutos")
                                        print("2. Plot datos en voltaje")
                                        print("3. Plot ambos")
                                        plot_archivo = str(opcionn)+','+str(t_grabacion)+','+str(t_lectura)+','+str(iniciar)+'('+str(fecha)+').png'
                                        nombre_archivo = str(opcionn)+','+str(t_grabacion)+','+str(t_lectura)+','+str(iniciar)+'('+str(fecha)+').csv'
                                        nombre_carpeta = str(opcionn)+','+str(t_grabacion)+','+str(t_lectura)+','+str(iniciar)+'('+str(fecha)+')'
                                        ruta_carpeta2 = os.path.join(ruta_carpeta, nombre_carpeta)
                                        os.makedirs(ruta_carpeta2)
                                        ruta_completa_csv = os.path.join(ruta_carpeta2, nombre_archivo)
                                        ruta_completa_plot = os.path.join(ruta_carpeta2, plot_archivo)
                                        plotteo = str(input("Que quieres plotear?:"))
                                        if plotteo == '1':
                                            fig, ax = plt.subplots()
                                            ax.plot(tiempo,vector)
                                            plt.savefig(ruta_completa_plot, dpi=300)

                                        plt.show()


                                        if plotteo == '1':
                                            with open(ruta_completa_csv, 'w', newline='') as archivo_csv:
                                                escritor_csv = csv.writer(archivo_csv)
                                                escritor_csv.writerow(['Lecturas ADC'])
                                                for fila in vector:
                                                    escritor_csv.writerow([fila])

                                        print(f'Se han guardado los archivos PNG y CSV correctamente en {ruta_completa_csv}')
                                        time.sleep(1)
                                        print("CONTINUAR?")
                                        print("1. Si")
                                        print("2. No")
                                        reiniciar = input("Seleccione una opcionn: ")


                                        if reiniciar=="1":
                                            arduino.close()
                                            time.sleep(2)
                                            main()
                                        if reiniciar =="2":
                                            print("Adios")
                                            arduino.close()
                                            time.sleep(2)


                    elif iniciar =="no":
                        print("Saliendo del programa...")
                        arduino.close()
                        break
                    else:
                        print("Error")


        elif opcionn == '2':
            print("Finalizado...")
            arduino.close()
            time.sleep(2)
            break
        else:
            print("Opción inválidaaa. Por favor, selecciona una opción válida.")
#
if __name__ == "__main__":
    main()